In [51]:
!pip install pypdf
!pip install joblib
!pip install pandas
!pip install scikit-learn
!pip install xlrd
!pip install youtube_transcript_api
!pip install openpyxl

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Defaulting to user installation because normal site-pac

In [74]:
# 2021 Canadian Election data
from pypdf import PdfReader
import re
import pandas as pd

# Data extraction
parties = ['liberal', 'conservative', 'ndp', 'green']

not_alpha_regex = r"[^a-zA-Z]"
not_main_chars_regex = r"[^a-zA-Z\s.]"
excess_spaces_regex = r"\s+"

party_labels = []
texts = []

def hasAlphabeticalChars(string):
    return len(re.sub(not_alpha_regex, "", string)) > 0

for i, party in enumerate(parties):
    reader = PdfReader(f"../data/2021/platforms/{party}.pdf")

    for page in reader.pages:
        raw_text = page.extract_text()
        cleaned_text = re.sub(not_main_chars_regex, "", raw_text)
        cleaned_text = re.sub(excess_spaces_regex, " ", cleaned_text)
        
        page_sentences = list(filter(hasAlphabeticalChars, cleaned_text.split('.')))
        
        # Appending sentences to list
        for passage in page_sentences:
            party_labels.append(i)
            texts.append(passage)

sentences_data = {'text': texts, 'label': party_labels}

sentences = pd.DataFrame(sentences_data)

In [88]:
from youtube_transcript_api import YouTubeTranscriptApi
import pandas as pd


# 2025 Canadian Election Data
videos = pd.read_excel('../data/2025/political-classification-model-videos.xlsx')
video_links = videos['url'].tolist()
transcript_fetcher = YouTubeTranscriptApi()
parties = []
party_classes = []
texts = []

party_ids = {'conservative': 0, 'liberal': 1, 'ndp': 2, 'green': 3}
party_names = ['conservative', 'liberal', 'ndp', 'green']

for index, row in videos.iterrows():
    video_id = row['url'].split("=")[1]
    
    video_transcript = transcript_fetcher.fetch(video_id)
    
    for snippet in video_transcript:
        text = snippet.text
        
        if '[' not in text and len(text) > 15:
            print(f"{row['party']}: {snippet.text}")
            
            texts.append(text)
            party_classes.append(party_ids[row['party']])
            parties.append(row['party'])
            
data = pd.DataFrame({'text': texts, 'label': party_classes, 'label_text': parties})

conservative: came here the great Tony Paris with $50
conservative: in his pocket he decided that he would
conservative: use his skills as a sheet metal worker
conservative: to go out and on his own and start a
conservative: business and here we are a half century
conservative: later his brilliant son Angelo and now
conservative: his kids are working here as well how
conservative: old is this little guy here it's It's
conservative: Marco and he's Marco is the gonna be
conservative: taking over CEO next year so um and uh
conservative: he is one his daughter Sophia this is
conservative: good what an incredible
conservative: multi-generational family story of
conservative: entrepreneurship this is the Canadian
conservative: promise right here let's give them a big
conservative: applause they were just uh telling me
conservative: about their incredible products the um
conservative: the heating and cooling systems they use
conservative: they even let me put in the um put in
conservative: so

In [78]:
import torch
import pandas as pd
from datasets import Dataset
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from transformers import DataCollatorWithPadding
from sklearn.model_selection import train_test_split

# Load tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Split dataset
train_texts, test_texts, train_labels, test_labels = train_test_split(
    data["text"].tolist(), data["label"].tolist(), test_size=0.2, random_state=42
)

# Tokenize dataset
def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, padding=True)

train_dataset = Dataset.from_dict({"text": train_texts, "label": train_labels}).map(tokenize_function, batched=True)
test_dataset = Dataset.from_dict({"text": test_texts, "label": test_labels}).map(tokenize_function, batched=True)

# Load pre-trained BERT model
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=4)

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    auto_find_batch_size=True,
    num_train_epochs=5,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer),
)

# Train the model
trainer.train()

# Save model
model.save_pretrained("./canadian_political_bert")
tokenizer.save_pretrained("./canadian_political_bert")

print("Model training complete and saved.")


Map: 100%|██████████| 2528/2528 [00:00<00:00, 8709.03 examples/s]
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/home/isss11/.local/lib/python3.10/site-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/tmp/ipykernel_6449/3565422778.py:36: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss
1,1.093000,1.055804
2,0.712000,0.977171
3,0.366700,1.691333
4,0.146100,2.095401
5,0.066400,2.267874


Model training complete and saved.


In [97]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

def predict_political_affiliation(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    inputs = {key: val.to(device) for key, val in inputs.items()}  # Move to GPU if available

    with torch.no_grad():
        print(inputs)
        
        outputs = model(**inputs)
    
    logits = outputs.logits
    predicted_class = torch.argmax(logits, dim=1).item()

    # Map class index to party
    return party_names[predicted_class]

print(predict_political_affiliation("I think we need to cut taxes and put more police on the streets to stop crime."))


{'input_ids': tensor([[ 101, 1045, 2228, 2057, 2342, 2000, 3013, 7773, 1998, 2404, 2062, 2610,
         2006, 1996, 4534, 2000, 2644, 4126, 1012,  102]], device='cuda:0'), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
       device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]],
       device='cuda:0')}
conservative
